In [2]:
import cv2

from cv2_tools.Selection import SelectorCV2
from conexion_a_sucesiones_de_imagenes.Video_controller_final import VideoController


In [ ]:
class BasicMovementDetector(VideoController):

    def __init__(self, video, stream, fps, dilate=True):
        # Constructor de la clase
        # Llama al constructor de la clase padre VideoController y pasa los parámetros recibidos
        VideoController.__init__(self, video, stream, fps, dilate)


    def run(self):
        first_frame = None

        for frame in self.manager_cv2:
            # Comprueba si es el primer fotograma o se ha producido una acción en el teclado
            if first_frame is None or self.manager_cv2.key_manager.action:
                self.manager_cv2.key_manager.action = False
                # Guarda el primer fotograma y pasa a la siguiente iteración
                first_frame = VideoController.frame_conversion(frame)
                continue

            # Helper selector to draw contours
            # Crea un objeto selector para dibujar los contornos en el fotograma
            selector = SelectorCV2(color=(0, 0, 200), peephole=False)

            gray_frame = VideoController.frame_conversion(frame)
            # 1º Absolute substraction
            # Realiza una sustracción absoluta entre el primer fotograma y el fotograma actual
            substraction = cv2.absdiff(first_frame, gray_frame)
            # 2º Set threshold
            # Aplica un umbral a la diferencia obtenida anteriormente
            threshold = cv2.threshold(substraction, 25, 255, cv2.THRESH_BINARY)[1]
            # It will help with some holes
            # Si el parámetro dilate es verdadero, se aplica una dilatación a la imagen umbralizada
            if self.dilate:
                threshold = cv2.dilate(threshold, None, iterations=2)
            # 3º Contour/blob detection
            # Detecta los contornos en la imagen umbralizada
            contours = VideoController.find_contours(threshold)

            for contour in contours:
                # Agrega los contornos al objeto selector
                selector.add_zone(contour)

            # Dibuja los contornos en el fotograma
            frame = selector.draw(frame)
            # Muestra las imágenes resultantes en ventanas separadas
            cv2.imshow("BasicMovementDetector", frame)
            cv2.imshow("Threshold", threshold)
            cv2.imshow("Substraction", substraction)

        # Cierra todas las ventanas al final del bucle
        cv2.destroyAllWindows()
